In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.ml.fpm import FPGrowth
from pyspark.sql.functions import split, col

# Initialize a Spark session
spark = SparkSession.builder.appName("Purchase_Record").getOrCreate()
record = spark.read.option("header", "true" )\
                                              .option("inferSchema", "true" )\
                                              .option('delimiter'," ")\
                                              .csv('/content/purchase_record.txt')

In [ ]:
record.show()

+---+---------+
|tid|    items|
+---+---------+
|  1|    A,B,C|
|  2|    D,E,F|
|  3|  A,C,D,E|
|  4|    B,C,F|
|  5|  A,C,D,F|
|  6|    B,D,E|
|  7|  A,B,C,E|
|  8|  C,D,E,F|
|  9|  A,C,E,F|
| 10|  B,C,D,E|
| 11|    A,B,F|
| 12|A,C,D,E,F|
| 13|    B,D,F|
| 14|  A,B,C,D|
| 15|    A,E,F|
| 16|    C,D,E|
| 17|  A,B,C,F|
| 18|B,C,D,E,F|
| 19|A,B,C,D,E|
| 20|      C,F|
+---+---------+
only showing top 20 rows



In [ ]:
record_array = record .withColumn("items_array", split(record.items, ",")).drop('items')
record_array.show()

+---+---------------+
|tid|    items_array|
+---+---------------+
|  1|      [A, B, C]|
|  2|      [D, E, F]|
|  3|   [A, C, D, E]|
|  4|      [B, C, F]|
|  5|   [A, C, D, F]|
|  6|      [B, D, E]|
|  7|   [A, B, C, E]|
|  8|   [C, D, E, F]|
|  9|   [A, C, E, F]|
| 10|   [B, C, D, E]|
| 11|      [A, B, F]|
| 12|[A, C, D, E, F]|
| 13|      [B, D, F]|
| 14|   [A, B, C, D]|
| 15|      [A, E, F]|
| 16|      [C, D, E]|
| 17|   [A, B, C, F]|
| 18|[B, C, D, E, F]|
| 19|[A, B, C, D, E]|
| 20|         [C, F]|
+---+---------------+
only showing top 20 rows



In [ ]:
fpGrowth = FPGrowth(itemsCol="items_array", minSupport=0.5, minConfidence=0.6)
model = fpGrowth.fit(record_array)

In [ ]:
model.freqItemsets.show()

+------+----+
| items|freq|
+------+----+
|   [F]|  34|
|[F, D]|  25|
|   [A]|  32|
|   [B]|  32|
|   [C]|  36|
|[C, D]|  26|
|   [E]|  33|
|[E, D]|  27|
|   [D]|  37|
+------+----+

